In [30]:
import csv
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.cluster import KMeans

In [31]:
# Load data from the train data csv file
Feature = []
Label = []
with open('data/train/train.csv', 'r') as f:
    reader = csv.reader(f)
    train_data = list(reader)
    # remove the first row
    train_data = train_data[1:]
    for row in train_data:
        Feature.append(np.array([float(x) for x in row[1:-1]]))
        Label.append(row[-1])  
Features = np.array(Feature)
Labels = np.array(Label)

In [32]:
# implement pca on 431 components
pca = PCA(n_components=431)
pca.fit(Features)
Features_pca = pca.transform(Features)
# implement lda on default number of components
lda = LinearDiscriminantAnalysis()
lda.fit(Features_pca, Labels)
Features_lda = lda.transform(Features_pca)
# implement k means algorithm and add a cluster label to each data point
k_means = KMeans(n_clusters=19, random_state=0).fit(Features_lda)
Features_k_means = k_means.labels_
Features_k_means = Features_k_means.reshape(-1,1)
Features_k_means = preprocessing.scale(Features_k_means)
Features_k_means = np.concatenate((Features_k_means, Features_lda), axis=1)
# implement logistic regression
logreg = LogisticRegression(max_iter = 10000)
logreg.fit(Features_k_means, Labels)
# Load data from the test data csv file
test_features = []
with open('data/test/test.csv', 'r') as f:
    reader = csv.reader(f)
    test_data = list(reader)
    # remove the first row
    test_data = test_data[1:]
    for row in test_data:
        test_features.append(np.array([float(x) for x in row[1:]]))
test_features = np.array(test_features)
test_features_pca = pca.transform(test_features)
test_features_lda = lda.transform(test_features_pca)
test_features_k_means = k_means.predict(test_features_lda)
test_features_k_means = test_features_k_means.reshape(-1,1)
test_features_k_means = preprocessing.scale(test_features_k_means)
test_features_k_means = np.concatenate((test_features_k_means, test_features_lda), axis=1)
# predict the test data
test_pred = logreg.predict(test_features_k_means)
# write the prediction to the submission file
with open('data/test/submission_56_results.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Id', 'Category'])
    for i in range(len(test_pred)):
        writer.writerow([i, test_pred[i]])